# 🏠 ML-Based Property Moderation System

## Notebook 1: Chuẩn bị dữ liệu training

Notebook này sẽ:
1. Kết nối MongoDB
2. Fetch dữ liệu properties
3. Feature engineering
4. Export training data

In [ ]:
# Install dependencies
!pip install pymongo pandas numpy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
from datetime import datetime

In [ ]:
# Connect to MongoDB
MONGODB_URI = "YOUR_MONGODB_URI"  # Thay bằng URI của bạn

client = MongoClient(MONGODB_URI)
db = client['QLChoThueTro']
properties_collection = db['properties']

print(f"✅ Connected to MongoDB")
print(f"📊 Total properties: {properties_collection.count_documents({})}")

In [ ]:
# Fetch properties data
properties = list(properties_collection.find({
    'status': {'$in': ['available', 'rented']},
    'price': {'$gt': 0},
    'area': {'$gt': 0}
}))

print(f"📦 Fetched {len(properties)} properties")

# Preview first property
if properties:
    print("\n🔍 Sample property:")
    sample = properties[0]
    for key in ['title', 'price', 'area', 'propertyType', 'address']:
        print(f"  {key}: {sample.get(key)}")

In [ ]:
# Extract features
def extract_features(property_doc):
    """Extract features từ property document"""
    
    # Location
    location = property_doc.get('location', {})
    coords = location.get('coordinates', [106.7, 10.8])
    
    address = property_doc.get('address', {})
    district = address.get('district', '')
    city = address.get('city', '')
    
    # District mapping
    district_map = {
        'Quận 1': 1, 'Quận 2': 2, 'Quận 3': 3, 'Quận 4': 4, 'Quận 5': 5,
        'Quận 6': 6, 'Quận 7': 7, 'Quận 8': 8, 'Quận 9': 9, 'Quận 10': 10,
        'Quận 11': 11, 'Quận 12': 12, 'Quận Bình Tân': 13,
        'Quận Bình Thạnh': 14, 'Quận Gò Vấp': 15, 'Quận Phú Nhuận': 16,
        'Quận Tân Bình': 17, 'Quận Tân Phú': 18, 'Thành phố Thủ Đức': 19
    }
    
    city_map = {
        'TP. Hồ Chí Minh': 1,
        'Thành phố Hồ Chí Minh': 1,
        'Hà Nội': 2,
        'Đà Nẵng': 3
    }
    
    type_map = {
        'phong-tro': 1,
        'nha-nguyen-can': 2,
        'can-ho': 3,
        'chung-cu-mini': 4,
        'homestay': 5
    }
    
    # Amenities
    amenities = property_doc.get('amenities', {})
    amenity_keys = ['wifi', 'ac', 'parking', 'kitchen', 'water', 'laundry', 'balcony', 'security']
    
    features = {
        # Target
        'price': property_doc.get('price', 0),
        
        # Location
        'longitude': coords[0] if len(coords) > 0 else 106.7,
        'latitude': coords[1] if len(coords) > 1 else 10.8,
        'district_encoded': district_map.get(district, 0),
        'city_encoded': city_map.get(city, 1),
        
        # Property info
        'area': property_doc.get('area', 20),
        'bedrooms': property_doc.get('bedrooms', 1),
        'bathrooms': property_doc.get('bathrooms', 1),
        'propertyType_encoded': type_map.get(property_doc.get('propertyType', 'phong-tro'), 1),
        
        # Amenities
        'amenity_wifi': 1 if amenities.get('wifi') else 0,
        'amenity_ac': 1 if amenities.get('ac') else 0,
        'amenity_parking': 1 if amenities.get('parking') else 0,
        'amenity_kitchen': 1 if amenities.get('kitchen') else 0,
        'amenity_water': 1 if amenities.get('water') else 0,
        'amenity_laundry': 1 if amenities.get('laundry') else 0,
        'amenity_balcony': 1 if amenities.get('balcony') else 0,
        'amenity_security': 1 if amenities.get('security') else 0,
        
        # Derived
        'amenity_count': sum(1 for k in amenity_keys if amenities.get(k)),
        'price_per_sqm': property_doc.get('price', 0) / max(property_doc.get('area', 20), 1)
    }
    
    return features

# Extract features cho tất cả properties
data_rows = []
for prop in properties:
    try:
        features = extract_features(prop)
        data_rows.append(features)
    except Exception as e:
        print(f"Error extracting features: {e}")
        continue

# Create DataFrame
df = pd.DataFrame(data_rows)

print(f"\n✅ Extracted features for {len(df)} properties")
print(f"\n📊 DataFrame shape: {df.shape}")
print(f"\n📋 Columns: {df.columns.tolist()}")

df.head()

In [ ]:
# Data cleaning
print("🧹 Cleaning data...")

# Remove outliers
print(f"Before: {len(df)} rows")

# Giá hợp lý: 500k - 100M
df = df[(df['price'] >= 500_000) & (df['price'] <= 100_000_000)]

# Diện tích hợp lý: 5m2 - 500m2
df = df[(df['area'] >= 5) & (df['area'] <= 500)]

# Price per sqm hợp lý: 10k - 1M
df = df[(df['price_per_sqm'] >= 10_000) & (df['price_per_sqm'] <= 1_000_000)]

print(f"After: {len(df)} rows")

# Fill missing values
df = df.fillna(0)

print(f"\n✅ Data cleaned")

In [ ]:
# Export to CSV
output_path = '/content/training_data.csv'
df.to_csv(output_path, index=False)

print(f"\n💾 Saved training data to: {output_path}")
print(f"📊 Shape: {df.shape}")
print(f"📋 Features: {df.columns.tolist()}")

# Statistics
print(f"\n📈 Price statistics:")
print(df['price'].describe())

print(f"\n📏 Area statistics:")
print(df['area'].describe())

# Download file
from google.colab import files
files.download(output_path)